In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [9]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    driver.maximize_window()
    
    
    # =====================================================================================
    # get the login icon and click on it 
# =====================================================================================
    try:
        # get the login button
        login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')))
        if login:
            driver.execute_script("arguments[0].scrollIntoView();", login)
            login.click()
        else:
            pass
    except Exception as e:
        print(f"No login tab found and the error is {e}") 
        
    # num cars
    try:
        cars = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="text-caption-r text-monochrome-900 hidden md:block md:w-[200px]"]/span'))).text.strip()
        if cars:
            tot_cars = int(cars.split(" ")[0])
            print(f"Found {tot_cars} number of cars")
        else:
            print("no cars found")
    except Exception as e:
        print("No cars found ")
    
    
    # result
    results = []
    car_count = 0
    while car_count < tot_cars: #total_cars
        
        try:
            page_cars= WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="relative aspect-[4/3] h-full w-full overflow-hidden"]')))
            for i in range(len(page_cars)): #total_cars
                # if car_count >= cars:
                #     break
                
                try:
                    page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="relative aspect-[4/3] h-full w-full overflow-hidden"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    # time.sleep(1)
                    page_cars[i].click()
                    time.sleep(2)

                    details = {}
                
                    # title --> done
                    try:
                        title =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/h1[@class="text-h6-m md:text-h4-m"]'))).text.strip().split(", ")
                        if title:
                            details['Title'] =title[0]
                            details['Price'] = title[1]
                        else:
                            details['Title'] = "na"
                            details['Price'] = "na"
                            
                    except Exception as e:
                        print("No car title") 
                    
                    # reg number --> done
                    try:
                        reg =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="text-footnote-m md:text-caption-m rounded bg-neutral-200 px-3 py-1"]'))).text.strip()  
                        if reg:
                            details['Reg'] = reg
                        else:
                            details['Reg'] = "na"
                    except Exception as e:
                        print("No reg found")
                        
                    # other info --> done
                    try:
                        info =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="flex flex-wrap gap-1 md:gap-3"]'))).text.strip() 
                        if info:
                            details['Information'] = info
                        else:
                            details['Information'] = 'na'
                    except Exception as e:
                        print("No information")  
                    
                    # imgs --> done
                    try:
                        img_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="flex -ml-4"])[2]')))
                        if img_card:
                            img_divs = WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="min-w-0 shrink-0 grow-0 pl-4 relative ml-2 aspect-[4/3] basis-1/3 overflow-hidden"]')))
                            if img_divs:
                                imgs_lst = []
                                for div in img_divs:
                                    imgs = div.find_element(By.TAG_NAME, 'img').get_attribute("src")
                                    imgs_lst.append(imgs)
                                details['Images'] = ", ".join(imgs_lst)
                    except Exception as e:
                        print("No images found")
                                    
                    
                    
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
                    # time.sleep(2)
                except Exception as e:
                    print("No cars to click")
                    break
        
                

        except Exception as e:
            print(f"No more cars to click {e}")
            break
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("laSource.csv", index=False)
    # time.sleep(5)
    driver.quit()
path = "https://www.sunmotors.co.uk/vehicle-search"
scrape(path)

Found 5563 number of cars
No car title
No reg found
No information
No images found
No cars to click
No more cars to click Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00CE0283+25139]
	(No symbol) [0x00C6B234]
	(No symbol) [0x00B40300]
	(No symbol) [0x00B7B1B8]
	(No symbol) [0x00BACDF6]
	(No symbol) [0x00BA87E5]
	(No symbol) [0x00BA7D66]
	(No symbol) [0x00B13245]
	(No symbol) [0x00B1379E]
	(No symbol) [0x00B13C1D]
	GetHandleVerifier [0x00FEB143+3215603]
	GetHandleVerifier [0x010022BA+3310186]
	GetHandleVerifier [0x00FFC4D2+3286146]
	GetHandleVerifier [0x00D79C80+654384]
	(No symbol) [0x00C745BD]
	(No symbol) [0x00B12F10]
	(No symbol) [0x00B12AD6]
	GetHandleVerifier [0x01076E5C+3788300]
	BaseThreadInitThunk [0x7552FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771E82AE+286]
	RtlGetAppContainerNamedObjectPath [0x771E827E+238]



In [10]:
df = pd.read_csv("laSource.csv")
df

,Title,Price,Reg,Information,Images
0,"2018 Peugeot 108 Active £7,195","£7,195",CU68YRT,"1L\nHatchback\nManual\nPetrol\n6,917 Miles\n2018",https://www.sunmotors.co.uk/_next/image?url=ht...
1,"2017 Ford Ka+ Zetec £7,895","£7,895",WN17XSP,"1.2L\nHatchback\nManual\nPetrol\n24,340 Miles\...",https://www.sunmotors.co.uk/_next/image?url=ht...
2,"2015 Peugeot 3008 Blue HDI S/S Allure £6,995","£6,995",PL15VBY,"1.6L\nSuv Estate\nManual\nDiesel\n47,622 Miles...",https://www.sunmotors.co.uk/_next/image?url=ht...
3,"2014 Citroen Berlingo Xtr £7,995","£7,995",NK64DNO,"1.6L\nVan Derived Car\nManual\nDiesel\n40,067 ...",https://www.sunmotors.co.uk/_next/image?url=ht...
4,Mazda Mazda6 2.0 SKYACTIV-G SE-L Tourer 5dr Pe...,"£7,995",CA65VXZ,"2L\nEstate\nManual\nPetrol\n65,293 Miles\n2016",https://www.sunmotors.co.uk/_next/image?url=ht...
5,Land Rover Range Rover Evoque 2.0 TD4 Autobiog...,"£16,995",GU16WDS,"2L\nSuv Estate\nAutomatic\nDiesel\n43,991 Mile...",https://www.sunmotors.co.uk/_next/image?url=ht...
6,Ford S-Max 1.6T EcoBoost Zetec MPV 5dr Petrol ...,"£6,995",GK63WHX,"1.6L\nMPV\nManual\nPetrol\n74,816 Miles\n2013",https://www.sunmotors.co.uk/_next/image?url=ht...
7,NaN,NaN,NaN,NaN,NaN


### image downloader

In [11]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Reg", "Images"]]

def download_images(data, main_folder="Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        image_urls = row["Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: Images\CU68YRT\CU68YRT_1_image_1.jpg
Downloaded: Images\CU68YRT\CU68YRT_2_image_2.jpg
Downloaded: Images\CU68YRT\CU68YRT_3_image_3.jpg
Downloaded: Images\CU68YRT\CU68YRT_4_image_4.jpg
Downloaded: Images\CU68YRT\CU68YRT_5_image_5.jpg
Downloaded: Images\CU68YRT\CU68YRT_6_image_6.jpg
Downloaded: Images\CU68YRT\CU68YRT_7_image_7.jpg
Downloaded: Images\CU68YRT\CU68YRT_8_image_8.jpg
Downloaded: Images\CU68YRT\CU68YRT_9_image_9.jpg
Downloaded: Images\CU68YRT\CU68YRT_10_image_10.jpg
Downloaded: Images\CU68YRT\CU68YRT_11_image_11.jpg
Downloaded: Images\CU68YRT\CU68YRT_12_image_12.jpg
Downloaded: Images\CU68YRT\CU68YRT_13_image_13.jpg
Downloaded: Images\CU68YRT\CU68YRT_14_image_14.jpg
Downloaded: Images\CU68YRT\CU68YRT_15_image_15.jpg
Downloaded: Images\CU68YRT\CU68YRT_16_image_16.jpg
Downloaded: Images\CU68YRT\CU68YRT_17_image_17.jpg
Downloaded: Images\CU68YRT\CU68YRT_18_image_18.jpg
Downloaded: Images\CU68YRT\CU68YRT_19_image_19.jpg
Downloaded: Images\CU68YRT\CU68YRT_20_image_20.jp